### Real Robot (Kochbot Tutorial)
#### Setup Kochbot's environment
    - conda create -n kochbot python=3.10 -y
    - conda activate kochbot
    - pip install numpy torch matplotlib dynamixel-sdk

#### Goal of this tutorial
1. Control robot with keyboard and inverse kinematics.
2. Save the joint position-based waypoint
3. Interpolate waypoints to make a trajectory
4. Save it as a data
5. Replay the trajectory data with different control gains.


In [1]:
import time
import numpy as np
import mujoco
import mujoco.viewer
import copy

from interface import SimulatedRobot
from robot import Robot

In [2]:
# port = '/dev/tty.usbmodem58760430141'
port = 'COM3'
real_robot = Robot(device_name=port)

disabling torque for servos [1, 2, 3, 4, 5, 6]


In [3]:
m = mujoco.MjModel.from_xml_path('low_cost_robot/scene.xml')
d = mujoco.MjData(m)
ee_name = 'joint6'

simulator = SimulatedRobot(m, d)


In [4]:
real_robot._disable_torque()

disabling torque for servos [1, 2, 3, 4, 5, 6]


In [5]:
real_robot._set_position_control()

disabling torque for servos [1, 2, 3, 4, 5, 6]
enabling torque for servos [1, 2, 3, 4, 5, 6]


In [ ]:
# Joint 3: 1.57 (90), Joint 5: 3.14 (180)
JOINT_OFFSETS = np.array([0, 0, -1.57, 0, 3.14, 0])

move_stride = 0.1
n_steps = 0
max_steps = 70
time_stride = 0.001
error_bound = 0.001
control_joint_idx = [0, 1, 2, 3, 4]
lr = 0.1
gripper_close = 2000
gripper_open = 3000

waypoints = []

print("Start Control Loop. Press 'q' to quit.")

while True:
    command = input("w/s/a/d/e/c (Move), q (Quit), f/r (Gripper), g (Save Waypoint): ")

    if command == 'q':
        break

    elif command in ['w', 's', 'a', 'd', 'e', 'c']:
        curr_pwm = real_robot.read_position()
        curr_pwm = np.array(curr_pwm)

        sim_pos = simulator._pwm2pos(curr_pwm)
        d.qpos[:6] = sim_pos + JOINT_OFFSETS
        
        mujoco.mj_step(m, d)
        ee_pos = simulator.read_ee_pos(joint_name=ee_name)
        
        ee_target_pos = copy.deepcopy(ee_pos)

        if command == 'w': ee_target_pos[1] += move_stride
        elif command == 's': ee_target_pos[1] -= move_stride
        elif command == 'a': ee_target_pos[0] -= move_stride
        elif command == 'd': ee_target_pos[0] += move_stride
        elif command == 'e': ee_target_pos[2] += move_stride
        elif command == 'c': ee_target_pos[2] -= move_stride

        print(f"Target EE: {ee_target_pos}")

        while True:

            q_target_pos = simulator.inverse_kinematics(ee_target_pos, lr, joint_name=ee_name)

            real_target_pos = q_target_pos - JOINT_OFFSETS[:len(q_target_pos)]
            
            q_target_pwm = simulator._pos2pwm(real_target_pos)
            q_target_pwm = np.array([int(q) for q in q_target_pwm])
            
            current_pwm_subset = curr_pwm[control_joint_idx]
            target_pwm_subset = q_target_pwm
            
            pwm_diff = target_pwm_subset - current_pwm_subset
            
            # speed control (default: -50, 50)
            pwm_diff = np.clip(pwm_diff, -5, 5) 
            
            safe_next_pwm = current_pwm_subset + pwm_diff
            curr_pwm[control_joint_idx] = safe_next_pwm

            real_robot.set_goal_pos(curr_pwm)

            # loop delay
            time.sleep(time_stride)

            d.qpos[:6] = simulator._pwm2pos(curr_pwm) + JOINT_OFFSETS
            mujoco.mj_step(m, d)
            
            ee_pos = simulator.read_ee_pos(joint_name=ee_name)
            error = np.linalg.norm(ee_target_pos - ee_pos)

            n_steps += 1
            if n_steps >= max_steps:
                print("Reached maximum number of steps.")
                break
            if error < error_bound:
                print("Target reached.")
                break
        n_steps = 0
        
    elif command in ['f', 'r']:
        target_val = gripper_close if command == 'f' else gripper_open
        curr_gripper = curr_pwm[-1]
        gripper_traj = np.linspace(curr_gripper, target_val, num=10)
        
        for pwm in gripper_traj:
            curr_pwm[-1] = int(pwm)
            real_robot.set_goal_pos(curr_pwm)
            time.sleep(time_stride)
            
    elif command == 'g':
        waypoint = real_robot.read_position()
        waypoints.append(waypoint)
        print("Saved waypoint:", waypoint)

NameError: name 'np' is not defined

### Play the saved waypoints

In [7]:
# go to the first waypoint
next_wp = waypoints[0]
curr_pos = real_robot.read_position()
curr_pos = np.array(curr_pos)
next_wp = np.array(next_wp)
n_steps = 0
max_steps = 1000

print(curr_pos, next_wp)
q_stride = 20
q_bound = 50

while n_steps < max_steps:
    delta_pos = next_wp - curr_pos
    error = np.linalg.norm(delta_pos)
    if error < q_bound:
        print("Reached the first waypoint.")
        break
    step_pos = curr_pos + (delta_pos / error) * q_stride
    step_pwm = np.array([int(p) for p in step_pos])
    real_robot.set_goal_pos(step_pwm)
    curr_pos = step_pwm
    n_steps += 1
    time.sleep(time_stride)

for wp in waypoints:
    print('Moving to waypoint:', wp)
    next_wp = np.array(wp)
    curr_pos = real_robot.read_position()
    curr_pos = np.array(curr_pos)
    n_steps = 0

    while n_steps < max_steps:
        delta_pos = next_wp - curr_pos
        error = np.linalg.norm(delta_pos)
        if error < q_bound:
            print("Reached the next waypoint.")
            break
        step_pos = curr_pos + (delta_pos / error) * q_stride
        step_pwm = np.array([int(p) for p in step_pos])
        real_robot.set_goal_pos(step_pwm)
        curr_pos = step_pwm
        n_steps += 1
        time.sleep(time_stride)


IndexError: list index out of range